In [ ]:
from Scripts.Othello.Board import Board
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Replay Memory

Nous utiliserons la mémoire de relecture de l'expérience pour former notre DQN. Elle stocke les transitions que l'agent observe, ce qui nous permet de réutiliser ces données plus tard. En prélevant des échantillons de façon aléatoire, les transitions qui constituent un lot sont décorrélées. Il a été démontré que cela stabilise et améliore grandement la procédure d'entraînement du DQN.

Pour cela, nous allons avoir besoin de deux classes :

- **Transition** - un tuple nommé représentant une seule transition dans notre environnement. Il fait essentiellement correspondre les paires (état, action) à leur résultat (next_state, récompense), l'état étant l'image de différence d'écran comme décrit plus loin.
- **ReplayMemory** - un tampon cyclique de taille limitée qui contient les transitions observées récemment. Il met également en œuvre une méthode .sample() pour sélectionner un lot aléatoire de transitions pour l'entraînement.


In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)